## 공공데이터 포털 데이터 Api 호출해보기

LAWD_CD = "강서구", "마포구", "송파구"  
DEAL_YMD = 202501 ~ 202503  

2중 반복

In [ ]:
# LAWD_CD 검색 주소: https://www.code.go.kr/stdcode/regCodeL.do

# 아파트 매매 실거래가 공개 자료(상세) Api 요청변수(Request Parameter)

# LAWD_CD: 법정동코드 앞 5자리 ex) 11590(동작구)
# DEAL_YMD: 조회할 계약월 ex) 202504
# serviceKey: 공공데이터 포털 국토교통부_아파트 매매 실거래가 상세 자료 API
# pageNo: 페이지번호
# numOfRows: 한 페이지 결과 수

In [ ]:
# !pip install bs4
# !pip install lxml --force -U

In [ ]:
import warnings
import requests # API 연결 라이브러리
from lxml import html # XML 파싱을 위한 라이브러리
from bs4 import BeautifulSoup, XMLParsedAsHTMLWarning
import requests, bs4
import pandas as pd
warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

def loadAptTrdData(LawdCdList, serviceKey):
    try:
        endPoint = "http://apis.data.go.kr/1613000/RTMSDataSvcAptTradeDev/getRTMSDataSvcAptTradeDev"
        aptData = []
        numOfRows = 10

        for i in range(len(LawdCdList)):
            # 페이지 계산하는 부분
            for DEAL_YMD in range(202501, 202504):
                LAWD_CD = LawdCdList[i]
                pageNo = 1
                aptTradePriceUrl = f"{endPoint}?serviceKey={serviceKey}&LAWD_CD={LAWD_CD}&DEAL_YMD={DEAL_YMD}&pageNo={pageNo}&numOfRows={numOfRows}"
                resp = requests.get(aptTradePriceUrl)
                htmlObj = resp.text
                bsObj = bs4.BeautifulSoup(htmlObj, features="xml")
                pageCnt = bsObj.find(name="totalCount").text
                tempPageNo = int((int(pageCnt) / 10)) + 1

                # 계산된 페이지로 데이터 추출하는 부분
                for itemPage in range(tempPageNo):
                    pageNo = itemPage
                    aptTradePriceUrl = f"{endPoint}?serviceKey={serviceKey}&LAWD_CD={LAWD_CD}&DEAL_YMD={DEAL_YMD}&pageNo={pageNo}&numOfRows={numOfRows}"
                    resp = requests.get(aptTradePriceUrl)
                    htmlObj = resp.text
                    bsObj = bs4.BeautifulSoup(htmlObj, features="xml")
                    item = bsObj.find_all(name="item")
                    for j in range(len(item)):
                        aptName = item[j].find(name='aptNm').text # 아파트 이름
                        aptBuildYear = item[j].find(name='buildYear').text # 아파트 설립년도
                        aptDealAmount = item[j].find(name='dealAmount').text # 아파트 거래금액
                        dealMonth = item[j].find(name='dealMonth').text
                        if len(dealMonth) == 1:
                            dealMonth = "0" + dealMonth
                        else:
                            dealMonth
                        dealDay = item[j].find(name='dealDay').text
                        if len(dealDay) == 1:
                            dealDay = "0" + dealDay
                        aptDealDate = item[j].find(name='dealYear').text + dealMonth + dealDay # 계약날짜
                        aptDealGbn = item[j].find(name='dealingGbn').text # 거래유형
                        aptExcUseAr = item[j].find(name='excluUseAr').text # 전용면적
                        aptBuyer = item[j].find(name='buyerGbn').text # 매수자
                        aptSler = item[j].find(name='slerGbn').text # 매도자
                        aptLeaseGbn = item[j].find(name='landLeaseholdGbn').text # 토지임대 아파트 여부
                        aptRgstDate = item[j].find(name='rgstDate').text # 등기일자
                        aptData.append([aptName,aptBuildYear,aptDealAmount,aptDealDate,aptDealGbn,aptExcUseAr,aptBuyer,aptSler,aptLeaseGbn,aptRgstDate])
        return aptData
    except Exception as e:
        print(str(e))

In [ ]:
LawdCdList = ["11500", "11440", "11710"]
# serviceKey = "serviceKey" # 공공데이터 인코딩 인증키
aptData = loadAptTrdData(LawdCdList, serviceKey)

In [ ]:
columnList = ["aptName", "buildYear", "dealAmount", "dealDate", "dealingGbn", "excluUseAr", "buyerGbn", "slerGbn", "landLeaseholdGbn", "rgstDate"]
pd.DataFrame(aptData, columns=columnList)

,aptName,buildYear,dealAmount,dealDate,dealingGbn,excluUseAr,buyerGbn,slerGbn,landLeaseholdGbn,rgstDate
0,청솔수(711-6),2007,"80,200",20250111,중개거래,84.92,개인,개인,N,
1,마곡우촌아파트,2022,"43,800",20250124,직거래,28.29,개인,법인,N,25.03.28
2,가양6단지,1992,"85,000",20250131,중개거래,58.65,개인,개인,N,
3,우장산힐스테이트,2005,"165,000",20250104,중개거래,126.65,개인,개인,N,25.03.21
4,가양2단지(성지),1992,"49,000",20250123,중개거래,34.44,개인,개인,N,
...,...,...,...,...,...,...,...,...,...,...
2955,2차한양아파트,1984,"166,000",20250304,직거래,146.96,개인,개인,N,25.03.05
2956,레이크팰리스,2006,"299,000",20250303,중개거래,116.19,개인,개인,N,
2957,장미1,1979,"355,000",20250303,중개거래,155.22,개인,개인,N,
2958,리센츠,2008,"270,000",20250303,중개거래,84.99,개인,개인,N,
